# CAVEAT
The following instruction **does not work for 3D plots**:
```
ax.set_aspect(aspect = 'equal')
```

# Import modules and load data

In [ ]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from skimage import io
import torch

# Allow the interactive rotation of 3D scatter plots in jupyter notebook
import sys    
import os    
file_name =  os.path.basename(sys.argv[0])
#print(file_name == 'ipykernel_launcher.py') # This basicaly asks whether this file is a jupyter notebook?
if __name__ == "__main__":
    if file_name == 'ipykernel_launcher.py': # Run only in .ipynb, not in exported .py scripts
        get_ipython().run_line_magic('matplotlib', 'notebook') # Equivalent to ''%matplotlib notebook', but it is also understood by .py scripts
        
import functions_data_processing    
from functions_data_processing import load_camera_params, get_variables_from_vertex_full_Dataframe, load_faces
import functions_plot
from functions_plot import plot_camera_and_vertices

###
### 2nd DataSet - RendersTowelWall2 (~ 12/07/18)
###
# Choose sequence and animation frame
sequence_name = 'TowelWall'
animation_frame = '00022'
dataset_number = '2'
group_number = '001'
reordered=0 # Still using Blender's order

# Load camera parameters
variables = load_camera_params(sequence_name = 'TowelWall', dataset_number = '2')
RT_matrix = variables['RT_matrix']
RT_extended = variables['RT_extended']
print(RT_extended)
camera_worldCoord_x = variables['camera_worldCoord_x']
camera_worldCoord_y = variables['camera_worldCoord_y']
camera_worldCoord_z = variables['camera_worldCoord_z']
Intrinsic_matrix = variables['Intrinsic_matrix']
print(Intrinsic_matrix)
Camera_proj_matrix = variables['Camera_proj_matrix']
print('Camera_proj_matrix:')
print(Camera_proj_matrix)

# Load vertex data
variables = get_variables_from_vertex_full_Dataframe(sequence_name=sequence_name, dataset_number=dataset_number, 
                                                     group_number=group_number, animation_frame=animation_frame,
                                                     RT_extended=RT_extended, reordered=reordered)

occlusion_mask_values = variables['occlusion_mask_values']
u = variables['u']
v = variables['v']
u_visible = variables['u_visible']
v_visible = variables['v_visible']
X_world = variables['X_world']
Y_world = variables['Y_world']
Z_world = variables['Z_world']
X_camera = variables['X_camera']
Y_camera = variables['Y_camera']
Z_camera = variables['Z_camera']
nX_world = variables['nX_world']
nY_world = variables['nY_world']
nZ_world = variables['nZ_world']

# Load face data
faces = load_faces(sequence_name = 'TowelWall', dataset_number = '2', verbose=1)

# Converting between coordinate systems (camera, world, film, pixel)

### World coordinates --> Pixel coordinates
### Note that the Camera projection matrix of camera_parameters.txt is not the one that sends camera coordinates to film coordinates, but rather, the one sending world coordinates to pixel coordinates.

In [ ]:
vertex_number = 1
u_pixel_projected, v_pixel_projected, homogeneous_factor = functions_data_processing.world_to_pixel_coordinates(
    X_world[vertex_number], Y_world[vertex_number], Z_world[vertex_number],
    Camera_proj_matrix = Camera_proj_matrix,
    u_original=u[vertex_number], v_original=v[vertex_number])

#### Note that the homogeneous factor changes from vertex to vertex

In [ ]:
vertex_number = 1
u_pixel_projected1, v_pixel_projected1, homogeneous_factor1 = functions_data_processing.world_to_pixel_coordinates(
    X_world[vertex_number], Y_world[vertex_number], Z_world[vertex_number],
    Camera_proj_matrix = Camera_proj_matrix,
    u_original=u[vertex_number], v_original=v[vertex_number], verbose=0)
print('Homogeneous factor for vertex ' + str(vertex_number) + ':', homogeneous_factor)

vertex_number = 2
u_pixel_projected2, v_pixel_projected2, homogeneous_factor2 = functions_data_processing.world_to_pixel_coordinates(
    X_world[vertex_number], Y_world[vertex_number], Z_world[vertex_number],
    Camera_proj_matrix = Camera_proj_matrix,
    u_original=u[vertex_number], v_original=v[vertex_number], verbose=0)
print('Homogeneous factor for vertex ' + str(vertex_number) + ':', homogeneous_factor)

### Pixel coordinates --> World coordinates

In [ ]:
vertex_number = 1
X_world_projected1, Y_world_projected1, Z_world_projected1= functions_data_processing.pixel_to_world_coordinates(
    u=u_pixel_projected1, v=v_pixel_projected1, homogeneous_factor=homogeneous_factor1, 
    Camera_proj_matrix=Camera_proj_matrix,
    Z_world_GT=Z_world[vertex_number],
    X_world_GT=X_world[vertex_number],
    Y_world_GT=Y_world[vertex_number])

In [ ]:
X_world_projected1, Y_world_projected1, Z_world_projected1= functions_data_processing.pixel_to_world_coordinates_knowing_Y(
    u=u_pixel_projected1, v=v_pixel_projected1, 
    C=Camera_proj_matrix,
    Y_world_GT=Y_world[vertex_number],
    X_world_GT=X_world[vertex_number],
    Z_world_GT=Z_world[vertex_number])

In [ ]:
device='cpu'
vertex_number = [1, 2, 2, 2]
Camera_proj_matrix_inv = inv(Camera_proj_matrix[:,:-1]) 
Camera_proj_matrix_inv_tensor = torch.from_numpy(Camera_proj_matrix_inv).to(device).double() 
u=torch.from_numpy(np.asarray([u_pixel_projected1, u_pixel_projected2, u_pixel_projected2, u_pixel_projected2])).to(device).double().view(-1, 1)
v=torch.from_numpy(np.asarray([v_pixel_projected1, v_pixel_projected2, u_pixel_projected2, u_pixel_projected2])).to(device).double().view(-1, 1)
uv = torch.cat((u, v), 1)
Y_world_GT=torch.from_numpy(np.asarray(Y_world[vertex_number])).to(device).double().view(-1, 1)
X_world_GT=torch.from_numpy(np.asarray(X_world[vertex_number])).to(device).double().view(-1, 1)
Z_world_GT=torch.from_numpy(np.asarray(Z_world[vertex_number])).to(device).double().view(-1, 1)
xyz_world_GT = torch.cat((X_world_GT, Y_world_GT, Z_world_GT), 1)
    
xyz_world_projected = functions_data_processing.pixel_to_world_of_mesh_of_clouds_knowing_Y(
    uv, Camera_proj_matrix, Camera_proj_matrix_inv_tensor, Y_world_GT, xyz_world_GT, 1)

### World coordinates --> Camera coordinates

In [ ]:
# Aplication to a single vertex at a time
print("Coordinates of vertex", vertex_number)
print("X, Y, Z world coordinates:\t %8.2f %8.2f %8.2f" % (X_world[vertex_number], Y_world[vertex_number], Z_world[vertex_number]))
(X_camera_of1vertex, Y_camera_of1vertex, Z_camera_of1vertex) = functions_data_processing.world_to_camera_coordinates(X_world[vertex_number], Y_world[vertex_number], Z_world[vertex_number], RT_extended)
print("X, Y, Z camera coordinates:\t %8.2f %8.2f %8.2f" % (X_camera_of1vertex, Y_camera_of1vertex, Z_camera_of1vertex))
print()

In [ ]:
# Application to the whole dataset
(X_camera, Y_camera, Z_camera) = functions_data_processing.world_to_camera_coordinates(X_world, Y_world, Z_world, RT_extended)

# Checking that the conversion was correct

#### Computing camera position in camera coordinates (which should be (0,0,0)) from RT matrix and camera position in world coordinates - It works

In [ ]:
print("Coordinates of the camera position")
print("X, Y, Z world coordinates:\t %8.2f %8.2f %8.2f" % (camera_worldCoord_x, camera_worldCoord_y, camera_worldCoord_z))
(camera_cameraCoord_x, camera_cameraCoord_y, camera_cameraCoord_z) = functions_data_processing.world_to_camera_coordinates(camera_worldCoord_x, camera_worldCoord_y, camera_worldCoord_z, RT_extended)
print("X, Y, Z camera coordinates:\t %8.2f %8.2f %8.2f" % (camera_cameraCoord_x, camera_cameraCoord_y, camera_cameraCoord_z))

#### 3D scatter plot of the vertices coloured by visibility
Including camera position

In [ ]:
# World coordinates
plot_camera_and_vertices(sequence_name=sequence_name,
                             dataset_number=dataset_number, 
                             group_number=group_number, 
                             animation_frame=animation_frame,
                         X=X_world, Y=Y_world, Z=Z_world,
                         occlusion_mask_values=occlusion_mask_values,
                         camera_x=camera_worldCoord_x,
                         camera_y=camera_worldCoord_y,
                         camera_z=camera_worldCoord_z,
                        title='world coordinates')

# Camera coordinates
plot_camera_and_vertices(sequence_name=sequence_name,
                             dataset_number=dataset_number, 
                             group_number=group_number, 
                             animation_frame=animation_frame,
                         X=X_camera, Y=Y_camera, Z=Z_camera,
                         occlusion_mask_values=occlusion_mask_values,
                         camera_x=camera_cameraCoord_x,
                         camera_y=camera_cameraCoord_y,
                         camera_z=camera_cameraCoord_z,
                        title='camera coordinates')

# Camera coordinates - swap axes
plot_camera_and_vertices(sequence_name=sequence_name,
                             dataset_number=dataset_number, 
                             group_number=group_number, 
                             animation_frame=animation_frame,
                         X=X_camera, Y=Y_camera, Z=Z_camera,
                         occlusion_mask_values=occlusion_mask_values,
                         camera_x=camera_cameraCoord_x,
                         camera_y=camera_cameraCoord_y,
                         camera_z=camera_cameraCoord_z,
                         swap_axes=1,
                        title='camera coordinates')

### World coordinates to Camera coordinates - on normal vectors
See the function data_loading.world_to_camera_coordinates_normals

In [ ]:
# Aplication to a single normal vector at a time
for vertex_number in range(2):
    print("Coordinates of vertex", vertex_number)
    print("nX, nY, nZ world coordinates:\t %8.2f %8.2f %8.2f" % (nX_world[vertex_number], nY_world[vertex_number], nZ_world[vertex_number]))
    (nX_camera_of1vertex, nY_camera_of1vertex, nZ_camera_of1vertex) = functions_data_processing.world_to_camera_coordinates_normals(nX_world[vertex_number], nY_world[vertex_number], nZ_world[vertex_number], RT_matrix)
    print("nX, nY, nZ camera coordinates:\t %8.2f %8.2f %8.2f" % (nX_camera_of1vertex, nY_camera_of1vertex, nZ_camera_of1vertex))
    print()

In [ ]:
# Application to the whole dataset
(nX_camera, nY_camera, nZ_camera) = functions_data_processing.world_to_camera_coordinates_normals(nX_world, nY_world, nZ_world, RT_matrix)

# Checking that the conversion was correct

#### Computing camera position in camera coordinates (which should be (0,0,0)) from RT matrix and camera position in world coordinates - It works

In [ ]:
print("Coordinates of the camera position")
print("X, Y, Z world coordinates:\t %8.2f %8.2f %8.2f" % (camera_worldCoord_x, camera_worldCoord_y, camera_worldCoord_z))
(camera_cameraCoord_x, camera_cameraCoord_y, camera_cameraCoord_z) = functions_data_processing.world_to_camera_coordinates(camera_worldCoord_x, camera_worldCoord_y, camera_worldCoord_z, RT_extended)
print("X, Y, Z camera coordinates:\t %8.2f %8.2f %8.2f" % (camera_cameraCoord_x, camera_cameraCoord_y, camera_cameraCoord_z))

#### 3D plot of the normals
Including camera position

In [ ]:
# World coordinates
functions_plot.plot_normal_vectors(sequence_name=sequence_name,
                        dataset_number=dataset_number, 
                        group_number=group_number, 
                        animation_frame=animation_frame, 
                        X=X_world, Y=Y_world, Z=Z_world,
                        nX=nX_world, nY=nY_world, nZ=nZ_world,
                        occlusion_mask_values=occlusion_mask_values,
                        camera_x=camera_worldCoord_x,
                        camera_y=camera_worldCoord_y,
                        camera_z=camera_worldCoord_z,
                        plot_camera=1)

# Camera coordinates
functions_plot.plot_normal_vectors(sequence_name=sequence_name,
                        dataset_number=dataset_number, 
                        group_number=group_number, 
                        animation_frame=animation_frame, 
                        X=X_camera, Y=Y_camera, Z=Z_camera,
                        nX=nX_camera, nY=nY_camera, nZ=nZ_camera,
                        occlusion_mask_values=occlusion_mask_values,
                        camera_x=camera_cameraCoord_x,
                        camera_y=camera_cameraCoord_y,
                        camera_z=camera_cameraCoord_z,
                        plot_camera=1)

# Camera coordinates - swap axes
functions_plot.plot_normal_vectors(sequence_name=sequence_name,
                        dataset_number=dataset_number, 
                        group_number=group_number, 
                        animation_frame=animation_frame, 
                        X=X_camera, Y=Y_camera, Z=Z_camera,
                        nX=nX_camera, nY=nY_camera, nZ=nZ_camera,
                        occlusion_mask_values=occlusion_mask_values,
                        camera_x=camera_cameraCoord_x,
                        camera_y=camera_cameraCoord_y,
                        camera_z=camera_cameraCoord_z,
                        plot_camera=1, swap_axes=1)